In [1]:
import os
from datetime import datetime
import time
import tensorflow as tf
import os
import numpy as np
import h5py
import random
from PIL import Image
import tensorflow as tf
import json
from model import Model

/home/egert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
def build_batch(path_to_tfrecords_file, num_examples, batch_size, shuffled):
    assert tf.gfile.Exists(path_to_tfrecords_file), '%s not found' % path_to_tfrecords_file

    filename_queue = tf.train.string_input_producer([path_to_tfrecords_file], num_epochs=None)
    #image, length, digits = read_and_decode(filename_queue)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
            serialized_example,
            features={
                'image': tf.FixedLenFeature([], tf.string),
                'length': tf.FixedLenFeature([], tf.int64),
                'digits': tf.FixedLenFeature([5], tf.int64)
            })

    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.multiply(tf.subtract(image, 0.5), 2)
    
#     image = tf.reshape(image, [128, 128, 3])
    
    image = tf.reshape(image, [64, 64, 3])
    image = tf.random_crop(image, [54, 54, 3])
    
    length = tf.cast(features['length'], tf.int32)
    digits = tf.cast(features['digits'], tf.int32)
    
    
    
    min_queue_examples = int(0.4 * num_examples)

    image_batch, length_batch, digits_batch = tf.train.batch([image, length, digits],
                                                                     batch_size=batch_size,
                                                                     num_threads=2,
                                                                     capacity=min_queue_examples + 3 * batch_size)
    return image_batch, length_batch, digits_batch

In [12]:
#Used for test the accuracy for both training and test process
class Evaluator(object):
    def __init__(self, path_to_eval_log_dir):
        self.summary_writer = tf.summary.FileWriter(path_to_eval_log_dir)


 
    def evaluate(self, path_to_checkpoint, path_to_tfrecords_file, num_examples, global_step):
        batch_size = 128
        num_batches = num_examples // batch_size
        needs_include_length = False

        with tf.Graph().as_default():
            image_batch, length_batch, digits_batch = build_batch(path_to_tfrecords_file,
                                                                         num_examples=num_examples,
                                                                         batch_size=batch_size,
                                                                         shuffled=False)
            length_logits, digits_logits = Model.inference(image_batch, drop_rate=0.0)
            length_predictions = tf.argmax(length_logits, axis=1)
            digits_predictions = tf.argmax(digits_logits, axis=2)

            if needs_include_length:
                labels = tf.concat([tf.reshape(length_batch, [-1, 1]), digits_batch], axis=1)
                predictions = tf.concat([tf.reshape(length_predictions, [-1, 1]), digits_predictions], axis=1)
            else:
                labels = digits_batch
                predictions = digits_predictions

            labels_string = tf.reduce_join(tf.as_string(labels), axis=1)
            predictions_string = tf.reduce_join(tf.as_string(predictions), axis=1)

            accuracy, update_accuracy = tf.metrics.accuracy(
                labels=labels_string,
                predictions=predictions_string
            )

            tf.summary.image('image', image_batch)
            tf.summary.scalar('accuracy', accuracy)
            tf.summary.histogram('variables',
                                 tf.concat([tf.reshape(var, [-1]) for var in tf.trainable_variables()], axis=0))
            summary = tf.summary.merge_all()

            with tf.Session() as sess:
                sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
                coord = tf.train.Coordinator()
                threads = tf.train.start_queue_runners(sess=sess, coord=coord)

                restorer = tf.train.Saver()
                restorer.restore(sess, path_to_checkpoint)

                for _ in range(num_batches):
                    sess.run(update_accuracy)

                accuracy_val, summary_val = sess.run([accuracy, summary])
                self.summary_writer.add_summary(summary_val, global_step=global_step)

                coord.request_stop()
                coord.join(threads)

        return accuracy_val



In [19]:
#build the model with the tensor flow
def multi_digit_model(input_x, y_len, y_digit, drop_rate):
    with tf.variable_scope('hidden1'):
        conv = tf.layers.conv2d(input_x, filters = 48, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 2, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden1 = dropout

    with tf.variable_scope('hidden2'):
        conv = tf.layers.conv2d(hidden1, filters = 64, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 1, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden2 = dropout

    with tf.variable_scope('hidden3'):
        conv = tf.layers.conv2d(hidden2, filters = 128, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 2, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden3 = dropout

    with tf.variable_scope('hidden4'):
        conv = tf.layers.conv2d(hidden3, filters = 160, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 1, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden4 = dropout

    with tf.variable_scope('hidden5'):
        conv = tf.layers.conv2d(hidden4, filters = 192, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 2, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden5 = dropout

    with tf.variable_scope('hidden6'):
        conv = tf.layers.conv2d(hidden5, filters = 192, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 1, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden6 = dropout

    with tf.variable_scope('hidden7'):
        conv = tf.layers.conv2d(hidden6, filters = 192, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 2, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden7 = dropout

    with tf.variable_scope('hidden8'):
        conv = tf.layers.conv2d(hidden7, filters = 192, kernel_size = [5, 5], padding = 'same')
        norm = tf.nn.relu(tf.layers.batch_normalization(conv))
        maxpooling = tf.layers.max_pooling2d(norm, pool_size = [2, 2], strides = 1, padding = 'same')
        dropout = tf.layers.dropout(maxpooling, rate = drop_rate)
        hidden8 = dropout

    with tf.variable_scope('flatten'):
        flatten = tf.reshape(hidden8, [-1, 4 * 4 * 192])


    with tf.variable_scope('hidden9'):
        hidden9 = tf.layers.dense(flatten, units = 3072, activation = tf.nn.relu)


    with tf.variable_scope('hidden10'):
        hidden10 = tf.layers.dense(hidden9, units = 3072, activation = tf.nn.relu)


    with tf.variable_scope('digit_length'):
        length = tf.layers.dense(hidden10, units = 7)


    with tf.variable_scope('digit1'):
        digit1 = tf.layers.dense(hidden10, units = 11)


    with tf.variable_scope('digit2'):
        digit2 = tf.layers.dense(hidden10, units = 11)


    with tf.variable_scope('digit3'):
        digit3 = tf.layers.dense(hidden10, units = 11)


    with tf.variable_scope('digit4'):
        digit4 = tf.layers.dense(hidden10, units = 11)


    with tf.variable_scope('digit5'):
        digit5 = tf.layers.dense(hidden10, units = 11)


    digits = tf.stack([digit1, digit2, digit3, digit4, digit5], axis = 1)

    length_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_len, logits = length))
    d1_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_digit[:, 0], logits = digits[:, 0, :]))
    d2_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_digit[:, 1], logits = digits[:, 1, :]))
    d3_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_digit[:, 2], logits = digits[:, 2, :]))
    d4_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_digit[:, 3], logits = digits[:, 3, :]))
    d5_ce = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels = y_digit[:, 4], logits = digits[:, 4, :]))

    total_loss = length_ce + d1_ce + d2_ce + d3_ce + d4_ce + d5_ce


    return length, digits, total_loss

In [20]:
#Build the training process

def train(path_to_train_tfrecords_file, num_train_examples, path_to_val_tfrecords_file, num_val_examples,
           path_to_train_log_dir, path_to_restore_checkpoint_file, training_options):
    batch_size = training_options['batch_size']
    initial_patience = training_options['patience']
    num_steps_to_show_loss = 1
    num_steps_to_check = 10

    with tf.Graph().as_default():
        image_batch, length_batch, digits_batch = build_batch(path_to_train_tfrecords_file,
                                                                     num_examples=num_train_examples,
                                                                     batch_size=batch_size,
                                                                     shuffled=True)

        _, _, loss = multi_digit_model(image_batch, length_batch, digits_batch, drop_rate=0.2)
        
        global_step = tf.Variable(0, name='global_step', trainable=False)
        learning_rate = tf.train.exponential_decay(training_options['learning_rate'], global_step=global_step,
                                                   decay_steps=training_options['decay_steps'], decay_rate=training_options['decay_rate'], staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(loss, global_step=global_step)

        tf.summary.image('image', image_batch)
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('learning_rate', learning_rate)
        summary = tf.summary.merge_all()

        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter(path_to_train_log_dir, sess.graph)
            evaluator = Evaluator(os.path.join(path_to_train_log_dir, 'eval/val'))
    
            #tf.global_variables_initializer()
            sess.run(tf.global_variables_initializer())
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            saver = tf.train.Saver()
            if path_to_restore_checkpoint_file is not None:
                assert tf.train.checkpoint_exists(path_to_restore_checkpoint_file), \
                    '%s not found' % path_to_restore_checkpoint_file
                saver.restore(sess, path_to_restore_checkpoint_file)
                print ('Model restored from file: %s' % path_to_restore_checkpoint_file)

            print ('Start training')
            patience = initial_patience
            best_accuracy = 0.0
            duration = 0.0

            while True:
                start_time = time.time()
                _, loss_val, summary_val, global_step_val, learning_rate_val = sess.run([train_op, loss, summary, global_step, learning_rate])
                duration += time.time() - start_time

                if global_step_val % num_steps_to_show_loss == 0:
                    examples_per_sec = batch_size * num_steps_to_show_loss / duration
                    duration = 0.0
                    print ('%s: step %d, loss = %f ' % (
                        datetime.now(), global_step_val, loss_val))

                if global_step_val % num_steps_to_check != 0:
                    continue

                summary_writer.add_summary(summary_val, global_step=global_step_val)


                path_to_latest_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'latest.ckpt'))
                accuracy = evaluator.evaluate(path_to_latest_checkpoint_file, path_to_val_tfrecords_file,
                                              num_val_examples,
                                              global_step_val)
                print ('Validation accuracy is= %f, best accuracy %f' % (accuracy, best_accuracy))

                if accuracy > best_accuracy:
                    path_to_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'model.ckpt'),
                                                         global_step=global_step_val)
                    print ('Save file to: %s' % path_to_checkpoint_file)
                    patience = initial_patience
                    best_accuracy = accuracy
                else:
                    patience -= 1


                if patience == 0:
                    break

            coord.request_stop()
            coord.join(threads)
            print ('Training progess is finished')

In [6]:
# #Build the training process

# def train(path_to_train_tfrecords_file, num_train_examples, path_to_val_tfrecords_file, num_val_examples,
#            path_to_train_log_dir, path_to_restore_checkpoint_file, training_options):
#     batch_size = training_options['batch_size']
#     initial_patience = training_options['patience']
#     num_steps_to_show_loss = 1
#     num_steps_to_check = 10

#     with tf.Graph().as_default():
#         image_batch, length_batch, digits_batch = build_batch(path_to_train_tfrecords_file,
#                                                                      num_examples=num_train_examples,
#                                                                      batch_size=batch_size,
#                                                                      shuffled=True)

#         _, _, loss = multi_digit_model(image_batch, length_batch, digits_batch, drop_rate=0.2)
        
#         global_step = tf.Variable(0, name='global_step', trainable=False)
#         learning_rate = tf.train.exponential_decay(training_options['learning_rate'], global_step=global_step,
#                                                    decay_steps=training_options['decay_steps'], decay_rate=training_options['decay_rate'], staircase=True)
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#         train_op = optimizer.minimize(loss, global_step=global_step)

#         tf.summary.image('image', image_batch)
#         tf.summary.scalar('loss', loss)
#         tf.summary.scalar('learning_rate', learning_rate)
#         summary = tf.summary.merge_all()

#         with tf.Session() as sess:
#             summary_writer = tf.summary.FileWriter(path_to_train_log_dir, sess.graph)
#             evaluator = Evaluator(os.path.join(path_to_train_log_dir, 'eval/val'))
    
#             tf.global_variables_initializer()
#             sess.run(tf.global_variables_initializer())
#             #coord = tf.train.Coordinator()
#             #threads = tf.train.start_queue_runners(sess=sess, coord=coord)

#             saver = tf.train.Saver()
#             if path_to_restore_checkpoint_file is not None:
#                 assert tf.train.checkpoint_exists(path_to_restore_checkpoint_file), \
#                     '%s not found' % path_to_restore_checkpoint_file
#                 saver.restore(sess, path_to_restore_checkpoint_file)
#                 print ('Model restored from file: %s' % path_to_restore_checkpoint_file)

#             print ('Start training')
#             patience = initial_patience
#             best_accuracy = 0.0
#             #duration = 0.0

#             while True:

#                 start_time = time.time()
#                 print("Here")
#                 _, loss_val, summary_val, global_step_val, learning_rate_val = sess.run([train_op, loss, summary, global_step, learning_rate])
#                 #duration += time.time() - start_time

#                 if global_step_val % num_steps_to_show_loss == 0:
#                     examples_per_sec = batch_size * num_steps_to_show_loss / duration
#                     duration = 0.0
#                     print ('%s: step %d, loss = %f ' % (
#                         datetime.now(), global_step_val, loss_val))
                
#                 print("here2")
#                 if global_step_val % num_steps_to_check != 0:
#                     continue

#                 summary_writer.add_summary(summary_val, global_step=global_step_val)


#                 path_to_latest_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'latest.ckpt'))
#                 accuracy = evaluator.evaluate(path_to_latest_checkpoint_file, path_to_val_tfrecords_file,
#                                               num_val_examples,
#                                               global_step_val)
#                 print ('Validation accuracy is= %f, best accuracy %f' % (accuracy, best_accuracy))

#                 if accuracy > best_accuracy:
#                     path_to_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'model.ckpt'),
#                                                          global_step=global_step_val)
#                     print ('Save file to: %s' % path_to_checkpoint_file)
#                     patience = initial_patience
#                     best_accuracy = accuracy
#                 else:
#                     patience -= 1


#                 if patience == 0:
#                     break

#             coord.request_stop()
#             coord.join(threads)
#             print ('Training progess is finished')

In [22]:
#Train the model 
#define the folder path
# train_tfrecords_file = './data/train.tfrecords'
# val_tfrecords_file = './data/val.tfrecords'
# tfrecords_meta_file = './data/meta.json'

tfrecords_meta_file = './toy_data/meta.json'
train_tfrecords_file = './toy_data/train.tfrecords'
val_tfrecords_file = './toy_data/val.tfrecords'


log_dir = './logs/toy_train'
restore_checkpoint_file = './logs/train/latest.ckpt'
opt = {
    'batch_size': 32,
    'learning_rate': 1e-2,
    'patience': 100,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, restore_checkpoint_file,
           opt)

INFO:tensorflow:Restoring parameters from ./logs/train/latest.ckpt
Model restored from file: ./logs/train/latest.ckpt
Start training
2019-06-10 23:48:17.983134: step 3111, loss = 20.713161 
2019-06-10 23:48:19.196619: step 3112, loss = 10.829849 
2019-06-10 23:48:20.417111: step 3113, loss = 12.689416 
2019-06-10 23:48:21.641151: step 3114, loss = 11.024817 
2019-06-10 23:48:22.862260: step 3115, loss = 10.206832 
2019-06-10 23:48:24.088994: step 3116, loss = 10.002885 
2019-06-10 23:48:25.502096: step 3117, loss = 9.729511 
2019-06-10 23:48:26.732488: step 3118, loss = 9.583133 
2019-06-10 23:48:27.962624: step 3119, loss = 9.522475 
2019-06-10 23:48:29.195771: step 3120, loss = 9.417605 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.000000
2019-06-10 23:48:38.100403: step 3121, loss = 9.964670 
2019-06-10 23:48:39.328769: step 3122, loss = 9.606777 
2019-06-10 23:48:40.556190: step 3123, loss = 9.926910 
2019-

2019-06-10 23:52:26.555387: step 3229, loss = 9.225046 
2019-06-10 23:52:27.873894: step 3230, loss = 9.631406 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.001953, best accuracy 0.001953
2019-06-10 23:52:36.731723: step 3231, loss = 9.333470 
2019-06-10 23:52:38.051072: step 3232, loss = 9.530392 
2019-06-10 23:52:39.371442: step 3233, loss = 9.321954 
2019-06-10 23:52:40.689970: step 3234, loss = 9.356114 
2019-06-10 23:52:41.994482: step 3235, loss = 10.075003 
2019-06-10 23:52:43.314804: step 3236, loss = 9.365946 
2019-06-10 23:52:44.632947: step 3237, loss = 9.494125 
2019-06-10 23:52:45.952233: step 3238, loss = 9.470393 
2019-06-10 23:52:47.271064: step 3239, loss = 9.454496 
2019-06-10 23:52:48.578678: step 3240, loss = 9.419609 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.001953
2019-06-10 23:52:57.570211: step 3241, loss = 9.544805 
2019-06-10 23:5

2019-06-10 23:56:36.903767: step 3348, loss = 9.499619 
2019-06-10 23:56:38.218540: step 3349, loss = 9.214917 
2019-06-10 23:56:39.524630: step 3350, loss = 9.609226 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.001953
2019-06-10 23:56:48.418315: step 3351, loss = 9.263879 
2019-06-10 23:56:49.733211: step 3352, loss = 9.425684 
2019-06-10 23:56:51.043389: step 3353, loss = 9.630630 
2019-06-10 23:56:52.359902: step 3354, loss = 9.503061 
2019-06-10 23:56:53.671234: step 3355, loss = 9.954441 
2019-06-10 23:56:54.977403: step 3356, loss = 9.466627 
2019-06-10 23:56:56.288298: step 3357, loss = 9.506738 
2019-06-10 23:56:57.592578: step 3358, loss = 9.528785 
2019-06-10 23:56:58.902089: step 3359, loss = 9.477210 
2019-06-10 23:57:00.221045: step 3360, loss = 9.684330 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.001953
2019-06-10 23:57

2019-06-11 00:00:47.040062: step 3467, loss = 9.540062 
2019-06-11 00:00:48.360441: step 3468, loss = 9.094721 
2019-06-11 00:00:49.678935: step 3469, loss = 9.311183 
2019-06-11 00:00:50.997763: step 3470, loss = 9.096273 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.001953
2019-06-11 00:00:59.885700: step 3471, loss = 9.139471 
2019-06-11 00:01:01.212359: step 3472, loss = 9.303454 
2019-06-11 00:01:02.537466: step 3473, loss = 9.430755 
2019-06-11 00:01:03.850478: step 3474, loss = 9.302498 
2019-06-11 00:01:05.174666: step 3475, loss = 9.705768 
2019-06-11 00:01:06.492217: step 3476, loss = 9.261005 
2019-06-11 00:01:07.811488: step 3477, loss = 9.421761 
2019-06-11 00:01:09.129186: step 3478, loss = 9.588011 
2019-06-11 00:01:10.440155: step 3479, loss = 9.509363 
2019-06-11 00:01:11.756070: step 3480, loss = 9.518348 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accura

2019-06-11 00:04:58.305351: step 3586, loss = 9.234676 
2019-06-11 00:04:59.617353: step 3587, loss = 9.646927 
2019-06-11 00:05:00.942817: step 3588, loss = 9.278429 
2019-06-11 00:05:02.257475: step 3589, loss = 9.505283 
2019-06-11 00:05:03.564737: step 3590, loss = 9.352655 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.001953
2019-06-11 00:05:12.371511: step 3591, loss = 9.400748 
2019-06-11 00:05:13.670330: step 3592, loss = 9.668475 
2019-06-11 00:05:14.980323: step 3593, loss = 9.475307 
2019-06-11 00:05:16.295167: step 3594, loss = 9.561904 
2019-06-11 00:05:17.603512: step 3595, loss = 9.628975 
2019-06-11 00:05:18.902263: step 3596, loss = 9.261583 
2019-06-11 00:05:20.191737: step 3597, loss = 9.511451 
2019-06-11 00:05:21.505923: step 3598, loss = 9.239698 
2019-06-11 00:05:22.818838: step 3599, loss = 9.179420 
2019-06-11 00:05:24.128985: step 3600, loss = 9.283172 
INFO:tensorflow:Restoring parame

...

Validation accuracy is= 0.000000, best accuracy 0.003906
2019-06-11 00:29:53.128356: step 4301, loss = 9.408414 
2019-06-11 00:29:54.434783: step 4302, loss = 9.222980 
2019-06-11 00:29:55.744394: step 4303, loss = 9.375636 
2019-06-11 00:29:57.062017: step 4304, loss = 9.309796 
2019-06-11 00:29:58.373259: step 4305, loss = 9.274045 
2019-06-11 00:29:59.682125: step 4306, loss = 9.375425 
2019-06-11 00:30:00.992399: step 4307, loss = 9.093883 
2019-06-11 00:30:02.288905: step 4308, loss = 9.075742 
2019-06-11 00:30:03.607526: step 4309, loss = 8.990395 
2019-06-11 00:30:04.918354: step 4310, loss = 9.437208 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.001953, best accuracy 0.003906
2019-06-11 00:30:13.729629: step 4311, loss = 9.227997 
2019-06-11 00:30:15.034469: step 4312, loss = 9.169665 
2019-06-11 00:30:16.333816: step 4313, loss = 9.310437 
2019-06-11 00:30:17.647578: step 4314, loss = 9.625814 
2019-06-11 00:30:18.957622: ste

Validation accuracy is= 0.000000, best accuracy 0.003906
2019-06-11 00:34:03.654563: step 4421, loss = 9.156550 
2019-06-11 00:34:04.958253: step 4422, loss = 9.117842 
2019-06-11 00:34:06.265762: step 4423, loss = 9.014236 
2019-06-11 00:34:07.578810: step 4424, loss = 9.574151 
2019-06-11 00:34:08.884613: step 4425, loss = 8.963925 
2019-06-11 00:34:10.195987: step 4426, loss = 9.618514 
2019-06-11 00:34:11.492694: step 4427, loss = 9.213699 
2019-06-11 00:34:12.792922: step 4428, loss = 9.462733 
2019-06-11 00:34:14.090786: step 4429, loss = 9.499313 
2019-06-11 00:34:15.385277: step 4430, loss = 9.139336 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.003906, best accuracy 0.003906
2019-06-11 00:34:24.249944: step 4431, loss = 9.193244 
2019-06-11 00:34:25.538515: step 4432, loss = 9.125702 
2019-06-11 00:34:26.836392: step 4433, loss = 9.282534 
2019-06-11 00:34:28.136193: step 4434, loss = 9.270511 
2019-06-11 00:34:29.439269: ste

Validation accuracy is= 0.001953, best accuracy 0.003906
2019-06-11 00:38:15.320689: step 4541, loss = 9.107147 
2019-06-11 00:38:16.620241: step 4542, loss = 8.942568 
2019-06-11 00:38:17.921207: step 4543, loss = 9.098937 
2019-06-11 00:38:19.226316: step 4544, loss = 9.678442 
2019-06-11 00:38:20.527285: step 4545, loss = 9.106538 
2019-06-11 00:38:21.835516: step 4546, loss = 9.273067 
2019-06-11 00:38:23.138204: step 4547, loss = 9.299868 
2019-06-11 00:38:24.444706: step 4548, loss = 9.315900 
2019-06-11 00:38:25.757799: step 4549, loss = 9.547627 
2019-06-11 00:38:27.064475: step 4550, loss = 9.149002 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.003906
2019-06-11 00:38:35.915828: step 4551, loss = 9.174995 
2019-06-11 00:38:37.220135: step 4552, loss = 9.483348 
2019-06-11 00:38:38.526395: step 4553, loss = 9.198260 
2019-06-11 00:38:39.830162: step 4554, loss = 9.429474 
2019-06-11 00:38:41.147945: ste

Validation accuracy is= 0.001953, best accuracy 0.003906
2019-06-11 00:42:22.515869: step 4661, loss = 9.345804 
2019-06-11 00:42:23.821710: step 4662, loss = 8.868628 
2019-06-11 00:42:25.125382: step 4663, loss = 8.970951 
2019-06-11 00:42:26.428958: step 4664, loss = 9.115766 
2019-06-11 00:42:27.734331: step 4665, loss = 8.688101 
2019-06-11 00:42:29.056216: step 4666, loss = 9.164063 
2019-06-11 00:42:30.357876: step 4667, loss = 9.215488 
2019-06-11 00:42:31.659993: step 4668, loss = 9.309269 
2019-06-11 00:42:32.959515: step 4669, loss = 9.457223 
2019-06-11 00:42:34.266564: step 4670, loss = 9.063506 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.001953, best accuracy 0.003906
2019-06-11 00:42:43.045380: step 4671, loss = 9.150424 
2019-06-11 00:42:44.342310: step 4672, loss = 9.635301 
2019-06-11 00:42:45.647884: step 4673, loss = 9.246965 
2019-06-11 00:42:46.953353: step 4674, loss = 9.378435 
2019-06-11 00:42:48.259710: ste

Validation accuracy is= 0.000000, best accuracy 0.003906
2019-06-11 00:46:29.343400: step 4781, loss = 9.358844 
2019-06-11 00:46:30.652885: step 4782, loss = 9.113113 
2019-06-11 00:46:31.957878: step 4783, loss = 9.378630 
2019-06-11 00:46:33.261673: step 4784, loss = 9.335953 
2019-06-11 00:46:34.569139: step 4785, loss = 9.259337 
2019-06-11 00:46:35.872878: step 4786, loss = 9.254368 
2019-06-11 00:46:37.170896: step 4787, loss = 9.268286 
2019-06-11 00:46:38.482420: step 4788, loss = 9.503729 
2019-06-11 00:46:39.783653: step 4789, loss = 9.339597 
2019-06-11 00:46:41.094107: step 4790, loss = 9.202192 
INFO:tensorflow:Restoring parameters from ./logs/toy_train/latest.ckpt
Validation accuracy is= 0.000000, best accuracy 0.003906
2019-06-11 00:46:49.868079: step 4791, loss = 9.257203 
2019-06-11 00:46:51.180035: step 4792, loss = 9.288412 
2019-06-11 00:46:52.488087: step 4793, loss = 8.916496 
2019-06-11 00:46:53.778413: step 4794, loss = 9.225040 
2019-06-11 00:46:55.092366: ste

In [ ]:
#Train the model 
#define the folder path
# train_tfrecords_file = './data/train.tfrecords'
# val_tfrecords_file = './data/val.tfrecords'
# tfrecords_meta_file = './data/meta.json'

train_tfrecords_file = './toy_data/train.tfrecords'
val_tfrecords_file = './toy_data/val.tfrecords'
tfrecords_meta_file = './toy_data/meta.json'

log_dir = './logs/train'
restore_checkpoint_file = None
opt = {
    'batch_size': 32,
    'learning_rate': 1e-2,
    'patience': 100,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, restore_checkpoint_file,
           opt)